In [97]:

#
# The Localizer binds the models together and controls the update cycle in its "update" method.
#

import numpy as np
import matplotlib.pyplot as plt
import random

from models import StateModel,TransitionModel,ObservationModel



class Localizer:
    def __init__(self, sm):

        self.__sm = sm

        self.__tm = TransitionModel(self.__sm)
        self.__om = ObservationModel(self.__sm)

        # change in initialise in case you want to start out with something else
        # initialise can also be called again, if the filtering is to be reinitialised without a change in size
        print("before initialise")
        self.initialise()

    # retrieve the transition model that we are currently working with
    def get_transition_model(self) -> np.array:
        return self.__tm

    # retrieve the observation model that we are currently working with
    def get_observation_model(self) -> np.array:
        return self.__om

    # the current true pose (x, h, h) that should be kept in the local variable __trueState
    def get_current_true_pose(self) -> (int, int, int):
        x, y, h = self.__sm.state_to_pose(self.__trueState)
        return x, y, h

    # the current probability distribution over all states
    def get_current_f_vector(self) -> np.array(float):
        return self.__probs

    # the current sensor reading (as position in the grid). "Nothing" is expressed as None
    def get_current_reading(self) -> (int, int):
        ret = None
        if self.__sense != None:
            ret = self.__sm.reading_to_position(self.__sense)
        return ret;

    # get the currently most likely position, based on single most probable pose
    def most_likely_position(self) -> (int, int):
        return self.__estimate

    ################################### Here you need to really fill in stuff! ##################################
    # if you want to start with something else, change the initialisation here!
    #
    # (re-)initialise for a new run without change of size
    def initialise(self):
        self.__trueState = random.randint(0, self.__sm.get_num_of_states() - 1)
        self.__sense = None
        self.__probs = np.ones(self.__sm.get_num_of_states()) / (self.__sm.get_num_of_states())
        self.__estimate = self.__sm.state_to_position(np.argmax(self.__probs))
    
    # add your simulator and filter here, for example    
        
        #self.__rs = RobotSimAndFilter.RobotSim( ...)
        #self.__HMM = RobotSimAndFilter.HMMFilter( ...)
        self.__rs = RobotSim(self.__trueState)
        self.__HMM = HMMFilter(self.__om, self.__tm, self.__probs)
    #
    #  Implement the update cycle:
    #  - robot moves one step, generates new state / pose
    #  - sensor produces one reading based on the true state / pose
    #  - filtering approach produces new probability distribution based on
    #  sensor reading, transition and sensor models
    #
    #  Add an evaluation in terms of Manhattan distance (average over time) and "hit rate"
    #  you can do that here or in the simulation method of the visualisation, using also the
    #  options of the dashboard to show errors...
    #
    #  Report back to the caller (viewer):
    #  Return
    #  - true if sensor reading was not "nothing", else false,
    #  - AND the three values for the (new) true pose (x, y, h),
    #  - AND the two values for the (current) sensor reading (if not "nothing")
    #  - AND the error made in this step
    #  - AND the new probability distribution
    #
    
    def ls(self):
        x, y = self.__sm.reading_to_position(self.__trueState)
        return []
    
    def update(self) -> (bool, int, int, int, int, int, int, int, int, np.array(1)) :
        # update all the values to something sensible instead of just reading the old values...
        # 
        
        temp = self.__tm.get_T()[self.__trueState]
        move = np.random.choice(range(len(temp)), p=temp)
        self.__rs.move(move)
        self.__trueState = move
        readings = self.__om.get_o_reading(self.__sm.state_to_reading(self.__trueState))
        readings = [readings[i][i] for i in range(0,len(readings),4)]
        readings = readings + [1-sum(readings)]
        print('Readings', readings)
        reading = np.random.choice(range(len(readings)), p=readings)
        if reading >= len(readings)-1:
            self.__sense = None
        else:
            self.__sense = reading
            self.__probs = self.__HMM.forward(self.__sense)
            self.__estimate = self.__sm.state_to_position(np.argmax(self.__probs))
            print(self.__probs)
            print(self.__estimate)
        
        # this block can be kept as is
        ret = False  # in case the sensor reading is "nothing" this is kept...
        tsX, tsY, tsH = self.__sm.state_to_pose(self.__trueState)
        srX = -1
        srY = -1
        if self.__sense != None:
            srX, srY = self.__sm.reading_to_position(self.__sense)
            ret = True
            
        eX, eY = self.__estimate
        
        
        
        # this should be updated to spit out the actual error for this step
        
        error = abs(eX-tsX)+abs(eY-tsY) 

        
        # if you use the visualisation (dashboard), this return statement needs to be kept the same
        # or the visualisation needs to be adapted (your own risk!)
        return ret, tsX, tsY, tsH, srX, srY, eX, eY, error, self.__probs



In [98]:

import random
import numpy as np

from models import TransitionModel,ObservationModel,StateModel

#
# Add your Robot Simulator here
#
class RobotSim:
    def __init__(self, current_state):
        self.current_state = current_state
        
    def move(self, new_state):
        current_state = new_state
        
        
#
# Add your Filtering approach here (or within the Localiser, that is your choice!)
#
class HMMFilter:
    def __init__(self, sm, tm, probs):
        self.sm = sm
        self.tm = tm
        self.probs = probs
        
    def forward(self, state):
        alpha = 1
        self.probs = alpha*self.sm.get_o_reading(state) @ self.tm.get_T_transp() @ self.probs
        return self.probs


In [99]:
# In this cell, you can write your own "main" to run and evaluate your 
# implementation without using the visualisation below

# 
states = StateModel( 4, 4)
loc = Localizer(states)
print('-------true posistion-------', loc.get_current_true_pose())
print('------- current reading-------', loc.get_current_reading())
print('-------most likely posistion-----', loc.most_likely_position())
print(loc.update())
print('-------true posistion-------', loc.get_current_true_pose())
print('------- current reading-------', loc.get_current_reading())
print('-------most likely posistion-----', loc.most_likely_position())

before initialise
-------true posistion------- (1, 2, 0)
------- current reading------- None
-------most likely posistion----- (0, 0)
Readings [0.025, 0.05, 0.1, 0.05, 0.025, 0.05, 0.05, 0.05, 0.025, 0.025, 0.025, 0.025, 0.0, 0.0, 0.0, 0.0, 0.4999999999999999]
[0.00052083 0.         0.         0.00052083 0.00078125 0.0015625
 0.         0.00104167 0.0015625  0.00208333 0.         0.003125
 0.00104167 0.00104167 0.         0.         0.00052083 0.
 0.00078125 0.00039062 0.00078125 0.00104167 0.00104167 0.00078125
 0.00078125 0.00078125 0.00104167 0.00104167 0.00104167 0.00078125
 0.0015625  0.         0.00078125 0.         0.00052083 0.00039062
 0.00052083 0.00052083 0.00039062 0.00039062 0.00052083 0.00039062
 0.00039062 0.00052083 0.00078125 0.00039062 0.00052083 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
(0, 2)
(True, 0, 2, 0, 0, 2, 0, 2, 0, array([0.00052083, 0